# Model For Efficiently summarising Videos and Audio Files

## Importing requried libraries

In [1]:
!pip install transformers torch librosa soundfile

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

## Loadind ASR (Automatic Speech Recogniser) model

*ASR UPDATES:*
- ASR default version does not accurately output the text translation of the audio file input
- input file type: .wav

In [3]:
import transformers
print(transformers.__version__)

4.42.4


In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import torch
import librosa


### Testing Data Import

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# import os
# import zipfile

# extracted_path = '/content/extracted_audio/'
# with zipfile.ZipFile('/content/drive/MyDrive/1 Key Rule to Prevent 50% of Your Chess Mistakes.wav.download.zip', 'r') as zip_ref:
#     zip_ref.extractall(extracted_path)

# audio_file_path = os.path.join(extracted_path,'/content/drive/MyDrive/1 Key Rule to Prevent 50% of Your Chess Mistakes.wav.download/1 Key Rule to Prevent 50% of Your Chess Mistakes.wav')

In [20]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
audio_file_path = '/content/extracted_audio/1 Key Rule to Prevent 50% of Your Chess Mistakes.wav.download/1 Key Rule to Prevent 50% of Your Chess Mistakes.wav'
#load file and preprocess
audio_input = librosa.load(audio_file_path, sr=16000)[0]
input_values = tokenizer(audio_input, return_tensors="pt").input_values

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)
print(transcription)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mod

["THERE IS ONE MAJOR FACTOR THAT IS ABSOLUTELY NACESSARY FOR YOU TO PROGRESS TO THE LAVEL OFF SIXTEEN HUNDRED AND LATER ON TWO THOUSAND AND YOU ABSOLUTELY CAN NOT MAKE IT THERE WE'V OUD UNDER STANDING AND KNOWING HOW DO THES THING CLEARLY AND STRANGELY ENOUGH CHEST CRATERS USUALLY DON'T TALK ABOUT THIS AT ALL OR THEY JUST NATIONLY WITHOUT EXPLAIN HOW YOU ACTUALLYIN PL MENT THE SATY LET'S TALK ABOUT THIS POSITION IT IS WHITE TO PLAY PLEAS THINK ABOUT THIS FOR A SECOND AND PEK A MOVE WHICH YOU WOULD PLAY IF YOU WOLL WHITE AT THES POIN THEAS PEAK A MOVE AND AFTER THAT WILL DIS CUSS THIS NOW IN THE ACTUAL GAME WHIT PLAYED PONT OF FORD AND USUALLY WHEN I ASK STUDENTS TO EXPLAIN ME THERE THINK IT PROTSUS THEY TELL ME SOMETHING LIKE THIS SO N THIS POSITION THEY REALIZE THAT THEY CAN GO WITHTHEI'R PONT TO O FORD AND THAT HE'S THE NIGHT THAT HE'S THE POEN THAT OF COURSE THEY REALIZE THAT BLACK CAN TAKE HIY FOR THEY GET A CALCULATE THEY LIND WHAT IF PONTAKES O GAK CAN RE CAPTURE AND I STELL ATTA

## Loading Summarization Model

In [21]:
from transformers import BartTokenizer, BartForConditionalGeneration

#load model and tokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

#tokenize input text which we got from the ASR
inputs = tokenizer.encode(transcription[0], return_tensors='pt', max_length=1024, truncation=True)

#generate summary
summary_ids = model.generate(inputs, max_length=1000, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

THERE IS ONE MAJOR FACTor THAT IS ABSOLUTELY NACESSARY FOR YOU TO PROGRESS TO THE LAVEL OFF SIXTEEN HUNDRED AND LATER ON TWO THOUSAND AND YOU ABSOLutELY CAN'T MAKE IT THERE WE'V OUD UNDER STANDING AND KNOWING HOW DO THES THING CLEARLY AND STRANGELY ENOUGH CHEST CRATERS USUALLY DON'T TALK ABOUT THIS AT ALL OR THEY JUST NATIONLY WITHOUT EXPLAIN HOW YOU ACTUALLYIN PL MENT THE SATY.
